In [1]:
import tensorflow as tf
import os

In [2]:
learning_rate_A = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=1e-4, decay_steps=5000, decay_rate=0.9)
optimizer_A = tf.optimizers.SGD(learning_rate=learning_rate_A , momentum=1e-5)
learning_rate_B = tf.keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate=1e-3, decay_steps=10000, decay_rate=0.8)
optimizer_B = tf.optimizers.Adam(learning_rate=1e-3)#learning_rate_B)

In [3]:
class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name='bias',shape=[self.dense.input_shape[-1]],initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b = True)
        return self.activation(z + self.biases)

In [4]:
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1"])
#with mirrored_strategy.scope():

input = tf.keras.layers.Input(shape=(3), name='input_layer1')
#flatten = tf.keras.layers.Flatten()(input)
dense1 = tf.keras.layers.Dense(8, activation='relu')
dense2 = tf.keras.layers.Dense(12, activation='relu')
dense3 = tf.keras.layers.Dense(16, activation='relu')
model_encoder = dense1(input)
model_encoder = dense2(model_encoder)
model_encoder = dense3(model_encoder)
model_down = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")#input1, input2,input3,input4,input5,input6,input7,input8,input9,input10
#model_down.summary()
#input_encoder = tf.keras.layers.Input(shape=(15), name='input_layer2')
input2 = tf.keras.layers.Input(shape=(16), name='input_layer2')
model_decoder = DenseTranspose(dense3, activation = 'relu')(input2)
model_decoder = DenseTranspose(dense2, activation = 'relu')(model_decoder)
model_decoder = DenseTranspose(dense1, activation = 'relu')(model_decoder)
model_up = tf.keras.Model(inputs=[input2], outputs=model_decoder,name = "decoder")

#model_encoder_decoder.summary()
input3 = tf.keras.layers.Input(shape=(16), name='input_layer3')
model_ann = tf.keras.layers.Dense(16, activation='relu')(input3)
model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(16, activation='relu')(model_ann)
model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(16, activation='relu')(model_ann)
#model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
#model_ann = tf.keras.layers.Dense(16, activation='relu')(model_ann)
#model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
#model_ann = tf.keras.layers.Dense(16, activation='relu')(model_ann)
#model_ann = tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None)(model_ann)
model_ann = tf.keras.layers.Dense(9, activation='softmax')(model_ann)
model_ANN = tf.keras.Model(inputs=[input3], outputs=model_ann,name = "ann")

input_full = tf.keras.layers.Input(shape=(3), name='input_layer4')
encoder_out = model_down(input_full)
decoder_out = model_up(encoder_out)
ann_out = model_ANN(encoder_out)
#model_encoder_decoder_ann = tf.keras.Model(inputs=[input_full],outputs=[decoder_out],name = 'encoder_decoder_ann')
model_encoder_decoder_ann = tf.keras.Model(inputs=[input_full],outputs=[decoder_out,ann_out],name = 'encoder_decoder_ann')
#optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=1e-5)

In [5]:
model_encoder_decoder_ann.trainable_weights[0][0]

<tf.Tensor: id=445, shape=(8,), dtype=float32, numpy=
array([ 0.5950753 , -0.70727044, -0.5508436 ,  0.7181662 , -0.5506649 ,
        0.3399188 ,  0.3317322 ,  0.03995538], dtype=float32)>

In [6]:
path = "magnetic_checkpoints_1127_1"
latest = tf.train.latest_checkpoint(path)
latest

'magnetic_checkpoints_1127_1/ckpt_dropout_model_{epoch}-2200'

In [7]:
checkpoint = tf.train.Checkpoint(optimizerA=optimizer_A,
                                 optimizerB=optimizer_B,
                                 model_encoder_decoder_ann=model_encoder_decoder_ann,
                                 model_down=model_down)
checkpoint.restore(latest)

In [8]:
model_encoder_decoder_ann.trainable_weights[0][0]

<tf.Tensor: id=830, shape=(8,), dtype=float32, numpy=
array([ 0.06707124, -0.06958992, -0.08077073,  0.07173584, -0.00723271,
       -0.05953319,  0.11224787,  0.03083349], dtype=float32)>